# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import requests

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [140]:
stocks = pd.read_csv('nasdaq.csv')
from secrets import API_KEY

In [158]:
stocks

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
5,AAME,Atlantic American Corporation Common Stock,2.630,0.0300,1.154%,53565812.0,United States,NaN,1474,Finance,Life Insurance
6,AAOI,Applied Optoelectronics Inc. Common Stock,2.370,-0.0700,-2.869%,67835429.0,United States,2013.0,138843,Technology,Semiconductors
11,ABEO,Abeona Therapeutics Inc. Common Stock,2.610,-0.0600,-2.247%,44828835.0,United States,NaN,232860,Health Care,Biotechnology: Pharmaceutical Preparations
13,ABIO,ARCA biopharma Inc. Common Stock,2.300,-0.0400,-1.709%,33143329.0,United States,NaN,36118,Health Care,Biotechnology: In Vitro & In Vivo Diagnostic S...
15,ABOS,Acumen Pharmaceuticals Inc. Common Stock,5.900,-0.2100,-3.437%,241459176.0,United States,2021.0,81748,Health Care,Biotechnology: Pharmaceutical Preparations
...,...,...,...,...,...,...,...,...,...,...,...
3776,ZIVO,Zivo Bioscience Inc. Common Stock,2.925,0.0250,0.862%,27552506.0,United States,NaN,3226,Health Care,Biotechnology: Pharmaceutical Preparations
3777,ZIVOW,Zivo Bioscience Inc. Warrants,0.610,-0.1900,-23.75%,0.0,United States,NaN,500,Health Care,Biotechnology: Pharmaceutical Preparations
3782,ZTAQU,Zimmer Energy Transition Acquisition Corp. Units,9.990,0.0100,0.10%,0.0,United States,2021.0,105,Industrials,Consumer Electronics/Appliances
3785,ZVSA,ZyVersa Therapeutics Inc. Common Stock,2.570,-0.4700,-15.461%,23340540.0,United States,2022.0,305963,Health Care,Biotechnology: Pharmaceutical Preparations


In [154]:
stocks['Last Sale'] = stocks['Last Sale'].str.replace('$', '')

AttributeError: Can only use .str accessor with string values!

In [144]:
stocks['Last Sale'] = stocks['Last Sale'].astype(float)

In [157]:
stocks = stocks[stocks['Last Sale'] < 15]
stocks = stocks[stocks['Last Sale'] > 0.3]

In [28]:
symbols = stocks['Symbol'].to_numpy()

In [117]:
# with open('symbols.txt', 'w') as f:
#     for symbol in symbols:
#         f.write(f"{symbol}\n")

In [125]:
# new_symbols = []
# with open('symbols.txt') as f:
#     new_symbols = f.read().splitlines()

In [126]:
# new_symbols[-1]

'ZYXI'

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [112]:
rv_columns =[
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/RV',
    'EV/RV Percentile',
    'RV Score',
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol in symbols[0:5]:
    api_call_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
    data = requests.get(api_call_url).json()
    if('PERatio' in data):
        pe_ratio = None if data['PERatio'] == 'None' else float(data['PERatio'])
        # Price-to-book ratio
        pb_ratio = None if data['PERatio'] == 'None' else float(data['PriceToBookRatio'])
        # Price-to-sales ratio
        ps_ratio = None if data['PERatio'] == 'None' else float(data['PriceToSalesRatioTTM'])
        # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
        ev_to_ebitda = None if data['PERatio'] == 'None' else float(data['EVToEBITDA'])
        # Enterprise Value divided by Revenue (EV/rev)
        ev_to_revenue = None if data['PERatio'] == 'None' else float(data['EVToRevenue'])
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    float(stocks[stocks['Symbol'] == symbol]['Last Sale'].values[0][1:]),
                    'N/A',
                    pe_ratio,
                    'N/A',
                    pb_ratio,
                    'N/A',
                    ps_ratio,
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_revenue,
                    'N/A',
                    'N/A',
                ], index = rv_columns
            ), ignore_index=True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score


In [110]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [92]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
1,AACIU,10.1300,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
2,AACIW,0.0744,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
4,AAL,16.6400,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


In [93]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
1,AACIU,10.1300,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
2,AACIW,0.0744,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
3,AADI,12.7500,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A
4,AAL,16.6400,N/A,None,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [95]:
for column in [
    'Price',
    'Price-to-Earnings Ratio',
    'Price-to-Book Ratio',
    'Price-to-Sales Ratio',
    'EV/EBITDA',
    'EV/RV',
]:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [97]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A
1,AACIU,10.1300,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A
2,AACIW,0.0744,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A
3,AADI,12.7500,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A
4,AAL,16.6400,N/A,2.582,N/A,1.893,N/A,28.19,N/A,-2.232,N/A,12.63,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [98]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/RV':'EV/RV Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] \
            = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,N/A
1,AACIU,10.1300,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,N/A
2,AACIW,0.0744,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,N/A
3,AADI,12.7500,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,N/A
4,AAL,16.6400,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [99]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
1,AACIU,10.1300,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
2,AACIW,0.0744,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
3,AADI,12.7500,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
4,AAL,16.6400,N/A,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [100]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(inplace=True, drop=True)
# rv_dataframe = rv_dataframe[:50]

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

In [102]:
# position_size = float(portfolio_size)/len(rv_dataframe.index)
position_size = 6000
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe.sort_values('Price')

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
2,AACIW,0.0744,80645,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
0,AACI,10.0900,594,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
1,AACIU,10.1300,592,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
3,AADI,12.7500,470,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
4,AAL,16.6400,360,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0


In [103]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/RV,EV/RV Percentile,RV Score
0,AACI,10.0900,594,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
1,AACIU,10.1300,592,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
2,AACIW,0.0744,80645,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
3,AADI,12.7500,470,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0
4,AAL,16.6400,360,2.582,60.0,1.893,60.0,28.19,60.0,-2.232,60.0,12.63,60.0,60.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: